# Secure Password Management and File Encryption in Python

This notebook demonstrates how to:

1. Hash a password and save it in a JSON file.
2. Encrypt the JSON file using Fernet symmetric encryption.
3. Decrypt the JSON file and load the hashed password.
4. Verify a password entered by the user against the hashed password.

We'll use the `bcrypt` library for password hashing and the `cryptography` library for file encryption.

In [2]:
import bcrypt
import json
from getpass import getpass
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
import base64
import os

## Key Generation for Encryption

We'll start by generating a key that we'll use for encrypting the JSON file. We're using a password-based key derivation function (PBKDF2) with HMAC and SHA-256. 

In [3]:
# Generate a key for encryption
password_provided = "password"  # This is input in the form of a string
password = password_provided.encode()  # Convert to type bytes
salt = os.urandom(16)
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000
)
key = base64.urlsafe_b64encode(kdf.derive(password))  # Can only use kdf once

cipher_suite = Fernet(key)

## Password Hashing and Saving

Next, we will hash a password entered by the user, and save it along with a salt to a JSON configuration file.


In [4]:
# Hash a password
password = getpass("Enter the password: ").encode('utf-8')
salt = bcrypt.gensalt()
hashed_password = bcrypt.hashpw(password, salt)

# Save hashed password and salt to JSON config file
config = {
    'hashed_password': hashed_password.decode('utf-8'),  # bytes to string
    'salt': salt.decode('utf-8')  # bytes to string
}

with open('config.json', 'w') as f:
    json.dump(config, f)

Enter the password: ········


## File Encryption

We then encrypt the JSON configuration file using the key we generated earlier.


In [5]:
# Encrypt the JSON file
with open('config.json', 'rb') as f:
    data = f.read()

encrypted_data = cipher_suite.encrypt(data)

with open('config.json', 'wb') as f:
    f.write(encrypted_data)

## File Decryption and Password Loading

We can now load the encrypted JSON file, decrypt it, and load the hashed password and salt.


In [6]:
# Later... load and decrypt the JSON file
with open('config.json', 'rb') as f:
    encrypted_data = f.read()

data = cipher_suite.decrypt(encrypted_data)

config = json.loads(data)

hashed_password = config['hashed_password'].encode('utf-8')  # string to bytes
salt = config['salt'].encode('utf-8')  # string to bytes


## Password Verification

Finally, we verify a password entered by the user against the hashed password.


In [7]:
# Checking the password
check_password = getpass("Enter the password to check: ").encode('utf-8')
if bcrypt.checkpw(check_password, hashed_password):
    print("It Matches!")
else:
    print("It Does not Match :(")


Enter the password to check: ········
It Matches!


# Conclusion

In this notebook, we demonstrated how to securely manage passwords and sensitive data in Python using password hashing, file encryption, and decryption. 

Please remember, this notebook is for demonstration purposes and does not consider other important aspects of security such as secure handling of encryption keys and secure storage of hashed passwords.
